In [1]:
# main.py

import numpy as np
import pandas as pd
from salascorp_descriptive.kolmogorov_widget import kolmogorov_widget  # Importar la función del archivo separado

# Crear un dataset de prueba
dataset = pd.DataFrame({
    'variable1': np.random.normal(50, 10, 100),  # Distribución normal
    'variable2': np.random.exponential(1, 100),  # Distribución exponencial
    'variable3': np.random.uniform(0, 100, 100), # Distribución uniforme
    'group': np.random.choice(['A', 'B'], 100)   # Grupo categórico
})

# Llamar a la función desde el archivo importado
kolmogorov_widget(dataset)

interactive(children=(Dropdown(description='Variable:', options=('variable1', 'variable2', 'variable3'), value…